# Computation of Charged BH Life Evolution

# Useful Formulae
$$\frac{dQ^*}{dt} = Q^* \frac{f(M,Q) - h(M,Q)}{M^3}$$
$$\frac{dM}{dt} = \frac{f(M,Q)}{M^2}$$

$$ Q^* = Q/M $$
$$ r_{\pm} = r_S\frac{1 \pm \sqrt{1-4r^2_Q/r^2_S}}{2}$$
 where $r_q^2 = Q^2$ in dimensionless units
$$ T_Q = \frac{r_+ - r_-}{4 \pi r^2_+}$$
Emission Rate:
$$ \frac{d^2N_{i,lm}}{dEdt} = \frac{1}{2 \pi} \frac{\Gamma_{s_i,lm}(E,M,Q_j)}{ e^{E^{'} /T} - (-1)^{2s_i} }$$

Mass Loss Rate:
$$ f(M,Q) -M^2 \frac{dM}{dt}= M^2 \int_0^{+\infty} \sum_{i} g_i \sum_{lm} \frac{d^2N_{i,lm}}{dEdt} dE  $$

$$x = Er_S$$

Energies integrated over $10^-5 \times T < E <10^5 \times T$

# Integrating via Eulers Method

$$ f_{n+1} = f_n +\frac{df}{dt}\delta t$$

In [7]:
##### Opening fM and hM tables #####

import os
import pandas as pd
import numpy as np


# Specify the folder path
path = os.getcwd()
path = '{}/{}'.format(path,'Charge_Page_Tables')
dataframes = {}
index = []

# Loop through the files in the folder
for filename in os.listdir(path):
    # Check if the file is a CSV file
    if filename.endswith('.txt'):
        # Create the full file path
        file_path = os.path.join(path, filename)
        
        # Read the txt file into a DataFrame
        df = pd.read_csv(file_path, delim_whitespace=True, header = 'infer', skiprows=0, index_col = 'mass/Q')
        
        # Use the filename (without extension) as the key for the DataFrame
        df_name = os.path.splitext(filename)[0]  # Remove the file extension
        index.append(df_name)
        dataframes[df_name] = df
        

# Accessing the created DataFrames
# for name, df in dataframes.items():
    # print(f"DataFrame: {name}")
    # print(df.head())  # Show the first few rows of each DataFrame
    # print()

hM = dataframes[index[0]]
fM = dataframes[index[1]]
print(index)

['hM_testM', 'fM_testM']


/var/folders/__/68q69_j51cj16zb24xlwg1kw0000gn/T/ipykernel_65263/2630145412.py:22: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(file_path, delim_whitespace=True, header = 'infer', skiprows=0, index_col = 'mass/Q')
/var/folders/__/68q69_j51cj16zb24xlwg1kw0000gn/T/ipykernel_65263/2630145412.py:22: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(file_path, delim_whitespace=True, header = 'infer', skiprows=0, index_col = 'mass/Q')


In [10]:
## Relevant functions to iterate through BH evolution

#Finds correct fM value and returns dM/dt

def closest_value(lst, target):
    return min(lst, key=lambda x: abs(float(x) - target))

# Rate of mass loss
def dMdt(M,Q, table):
    header = table.columns
    row = table.index
    closest_Q = float(closest_value(header, Q))
    closest_m = float(closest_value(row,M))
    cell = table.loc[closest_m, "{:.5e}".format(closest_Q)]
    return -float(cell)/M**2


#Finds correct hM value and returns dQ^*/dt
def dQdt(M,Q, ftable,htable):
    fheader = ftable.columns
    frow = ftable.index
    hheader = htable.columns
    hrow = htable.index
        
    fclosest_Q = float(closest_value(fheader, Q))
    fclosest_m = float(closest_value(frow,M))
    cellf = ftable.loc[fclosest_m, "{:.5e}".format(fclosest_Q)]

    hclosest_Q = float(closest_value(hheader, Q))
    hclosest_m = float(closest_value(hrow,M))
    cellh = htable.loc[hclosest_m, "{:.5e}".format(hclosest_Q)]

    return Q*(float(cellf)-float(cellh))/M**3
#Rate of effective charge loss

#Function that allows time evolution of a charged black hole 
def evolution_times(t0,M0,Mrem,Q0,fM,hM,it_max):
    convfact = 	5.62e23 # Grams to GeV
    mass = []
    charge = []
    evol_times = []
    dt = 1.e-30 #(initial timestep in s)
    time = dt
    Mevol = M0*convfact #Initial mass in GeV
    Mrem = Mrem
    charge_evol = Q0 # Initial effective charge
    counter = 0
    print(time)
    evol_times.append(t0) #Initial timestep
    mass.append(Mevol)
    charge.append(Q0)

    while Mevol > Mrem:
        estimate_M = Mevol + dMdt(Mevol,charge_evol,fM)*dt
        estimate_Q = charge_evol + dQdt(Mevol,charge_evol,fM,hM)*dt
        evol_M = estimate_M/Mevol

        if charge_evol != 0:
            evol_param = estimate_Q/charge_evol

        else:
            evol_param = 0

        if evol_M < 0.9 or (evol_param != 0 and (evol_param <1.0 and evol_param < 0.9) or (evol_param >1.1 and evol_param > 1.0)): # one of the variations, M or Q is too large, decrease the timestep:
            dt =dt/2
            continue

        if evol_M > 0.99 and (dt/float(time) < 1.0) and ( evol_param == 0 or ((evol_param <= 1.0 and evol_param > 0.99) or (evol_param > 1.0 and evol_param < 1.01) )): # all variations are too low or the timestep does not become greater than the most recent time
            dt = dt*2
            continue

        else: # If rate of change of M and Q are both 1% < x <10%, the function is allowed to record the instance. This accounts for too large timesteps
            time = time + dt
            evol_times.append(time)
            Mevol = estimate_M 
            # print(estimate_M)
            # print(dt)
            # print()
            if estimate_Q < 1e-3:
                charge_evol = 0
            else:
                charge_evol = estimate_Q
            mass.append(Mevol)
            charge.append(charge_evol)
            counter += 1
        if counter > it_max:
            print('Max number of iterations reached. Raise iteration limit.')
            break
    return evol_times,mass,charge, counter

def txtgenerator(t,M,Q,count, filename):
    convfact = 	5.62e23 # Grams to GeV
    f = open(filename, "a")

    if os.path.exists(filename) == True:
        while True:

            inp = input('Overwrite data file (y/n) (BE CAREFUL TO USE CORRECT METRIC): ')
            
            if inp == 'y':
                open(filename, "w").close()
                f.write('Evolution of the BH masses and spins as functions of time. \n Total number of time iterations: {} \n\n'.format(count))
                f.write("t\tM\tQ\t\n")
                for time, mass, charge in zip(t, M, Q):
                    f.write(f"{time}\t{mass/convfact}\t{charge}\t\n")
                f.close()
                break
            elif t == 'n':
                break
            
            else:
                print('Invalid input')
            break    
    else:
        open(filename, "w").close()
        f.write('Evolution of the BH masses and spins as functions of time. \n Total number of time iterations: {} \n\n'.format(count))
        f.write("t\tM\tQ\t\n")
        for time, mass, charge in zip(t, M, Q):
            f.write(f"{time}\t{mass/convfact}\t{charge}\t\n")
        f.close()

In [12]:
## Defining intial parameters relevant to time evolution of BH 

M = 1e17 # Mass in grams
Q = 0.7  #Effective charge parameter, unitless defined as Q/M where Q is full charge (in units of energy) of black hole
Mp =1 # input the planck mass here
Mrem = 1.1*1.220890e19
t0 = 0 
alpha = 0.007297 # fine structure constant
e = np.sqrt(4*np.pi*alpha)
it_max = 5000 # maximum number of iterations, hard stopping point for computational time
print(e)
t,m,q,count = evolution_times(t0,M,Mrem,Q,fM,hM,it_max)


filename = 'Charge_evolution.txt'
txtgenerator(t,m,q,count, filename)






0.3028148054058435
1e-30
